## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN','NAME'], axis=1, inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

threshold = 10000
rare_applications = application_counts[application_counts < threshold].index

application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(rare_applications, 'Other')



APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 100000
application_types_to_replace = application_counts[application_counts < cutoff].index.tolist()

print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

['T3', 'T4', 'T6', 'T5', 'T19', 'T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


APPLICATION_TYPE
Other    34299
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

cutoff = 100000
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

CLASSIFICATION
Other    34299
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()

cutoff = 100000
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

['Other']


CLASSIFICATION
Other    34299
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)


In [10]:
print(application_df.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Co-operative', 'ORGANIZATION_Corporation',
       'ORGANIZATION_Trust', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_Y'],
      dtype='object')


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df['STATUS'].values  # Target array (prediction for)
X = application_df.drop('STATUS', axis=1).values  # Feature array (use to predict it with)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=23))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# First hidden layer
nn_model.add(Dense(units=64, activation='relu', input_shape=(23,)))

# Second hidden layer
nn_model.add(Dense(units=32, activation='relu'))

# Output layer
nn_model.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                384       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
 dense_3 (Dense)             (None, 64)                128       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,914
Trainable params: 2,914
Non-trainabl

In [14]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
X_train_scaled = X_train_scaled.reshape(-1, 23)
X_train_scaled = np.array(X_train_scaled, dtype='float32')
y_train = np.array(y_train, dtype='float32')


In [16]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, batch_size=32)

Epoch 1/100
686/686 [==============================] - 4s 4ms/step - loss: 0.0530 - accuracy: 0.9998 - val_loss: 5.5067e-04 - val_accuracy: 1.0000
Epoch 2/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 2.5665e-04 - val_accuracy: 1.0000
Epoch 3/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 1.7016e-04 - val_accuracy: 1.0000
Epoch 4/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 1.7431e-04 - val_accuracy: 1.0000
Epoch 5/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 1.4580e-04 - val_accuracy: 1.0000
Epoch 6/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 9.4575e-05 - val_accuracy: 1.0000
Epoch 7/100
686/686 [==============================] - 2s 2ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 3.350

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 2.2237e-04 - accuracy: 1.0000 - 603ms/epoch - 3ms/step
Loss: 0.0002223663468612358, Accuracy: 1.0


In [18]:
# Export our model to HDF5 file
nn_model.save("trained_model.h5")

The purpose of this analysis is to train a model for Alphabet Soup. The model should assist in finding which of the over 34,000 applicants has the highest chance of success in their ventures. 
The results of the analysis performed show an accuracy score of 1.00 and a very low loss. This indicates a high functioning model. 
In the data processing step we identified 'Status' as the target for the model. The features for the model included everything besides status, after dropping the EIN and NAME columns. 
We selected 2914 neurons, 2 hidden layers, 4 total layers, and relu and sigmoid activation functions for this neural network model. We selected these measurements to try and achieve top model performance. In order to make model performance even better we trained and fit the model, running 100 epochs to evaluate the model's loss and accuracy.
In summary the model that was trained for Alphabet soup was able to work through over 34000 records with a very high level of accuracy with minimal loss. 